# Starting and monitoring AI Platform Training jobs.

In [1]:
import base64
import os
import json
import time
import numpy as np
import tensorflow as tf

import google.auth

from google.auth.credentials import Credentials
from google.auth.transport.requests import AuthorizedSession

from typing import List, Optional, Text, Tuple


## Setting the notebook parameters

In [2]:
CAIP_ENDPOINT = "us-central1-aiplatform.googleapis.com"
CAIP_REGION = "us-central1"
TENSORBOARD_NAME = "demo-tensorboard"

## Preparing Managed TensorBoard 

### Create an authorized session

In [22]:
credentials, project_id = google.auth.default()
authed_session = AuthorizedSession(credentials)

caip_api_prefix = f'https://{CAIP_ENDPOINT}/v1alpha1/projects/{project_id}/locations/{CAIP_REGION}'

### Create a TensorBoard resources

In [23]:
api_url = f'{caip_api_prefix}/tensorboards'

request_body = {
    "display_name": TENSORBOARD_NAME
}

response = authed_session.post(api_url, data=json.dumps(request_body))
response.json()

{'name': 'projects/895222332033/locations/us-central1/tensorboards/7024208043814420480/operations/1694551927565582336',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.CreateTensorboardOperationMetadata',
  'genericMetadata': {'createTime': '2020-11-23T23:19:52.693614Z',
   'updateTime': '2020-11-23T23:19:52.693614Z'}}}

### List all tensorboards with a set name

In [24]:
api_url = f'{caip_api_prefix}/tensorboards?filter=display_name={TENSORBOARD_NAME}'

response = authed_session.get(api_url)
response.json()

{'tensorboards': [{'name': 'projects/895222332033/locations/us-central1/tensorboards/5236278991748333568',
   'displayName': 'demo-tensorboard',
   'createTime': '2020-11-23T23:18:52.933004Z',
   'updateTime': '2020-11-23T23:18:53.057311Z',
   'etag': 'AMEw9yMdG_TtJVIxyM1hiIOShO3Dv56z2xkhrh0Kyx6bY4Co6az0Zkg9BShGMcQYIdE='},
  {'name': 'projects/895222332033/locations/us-central1/tensorboards/7024208043814420480',
   'displayName': 'demo-tensorboard',
   'createTime': '2020-11-23T23:19:52.693614Z',
   'updateTime': '2020-11-23T23:19:52.891089Z',
   'etag': 'AMEw9yNR3ZgB_szVbo0ydVp8pA-byJLdfbzBIyLDTzvlUGePJC2BN8cofVVXUVo1-5j8'}]}

## Cleaning up

### List all tensorboards in the project

In [25]:
api_url = f'{caip_api_prefix}/tensorboards'

response = authed_session.get(api_url)
response.json()

{'tensorboards': [{'name': 'projects/895222332033/locations/us-central1/tensorboards/1215127474459901952',
   'displayName': 'tensorboard-1',
   'createTime': '2020-11-19T15:34:52.470258Z',
   'updateTime': '2020-11-19T15:34:52.601288Z',
   'etag': 'AMEw9yMQ9nP3PEDWq4stSpOLyE82wzUvCaGrBLzKW1w0IxiyBA2NRl8ho19RlYJmFty-'},
  {'name': 'projects/895222332033/locations/us-central1/tensorboards/5236278991748333568',
   'displayName': 'demo-tensorboard',
   'createTime': '2020-11-23T23:18:52.933004Z',
   'updateTime': '2020-11-23T23:18:53.057311Z',
   'etag': 'AMEw9yOiSaoPMMPtBSA5NNCL3FF7I5vdSiIRogUGR9h5TgpA5bEc2e7lnFIsF4xw5fg='},
  {'name': 'projects/895222332033/locations/us-central1/tensorboards/7024208043814420480',
   'displayName': 'demo-tensorboard',
   'createTime': '2020-11-23T23:19:52.693614Z',
   'updateTime': '2020-11-23T23:19:52.891089Z',
   'etag': 'AMEw9yN1qeePzJpcAYwyggzg_Sq3KFG2hhYO0Y1_M7i7CWWPF3BwNcW7y4iLZf0v3xHX'}]}

### Delete a TensorBoard resource

In [20]:
tensorboard_id = '4655314639817539584'

api_url = f'{caip_api_prefix}/tensorboards/{tensorboard_id}'

response = authed_session.delete(api_url)
response.json()

{'error': {'code': 404,
  'message': 'The tensorboard projects/jk-mlops-dev/locations/us-central1/tensorboards/4655314639817539584 does not exist.',
  'status': 'NOT_FOUND'}}